# NameX Six Months Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

Six month totals before running time.

In [ ]:
%%sql stat_six_month_completed  <<
SELECT e.user_id     
     , (select username from users u where u.id=e.user_id) AS EXAMINER
     , count(e.*) FILTER (WHERE e.state_cd = 'APPROVED')  AS APPROVED
     , count(e.*) FILTER (WHERE e.state_cd = 'REJECTED')  AS REJECTED
     , count(e.*) FILTER (WHERE e.state_cd = 'CONDITIONAL')  AS CONDITIONAL
     , count(e.*) FILTER (WHERE e.state_cd = 'CANCELLED')  AS CANCELLED
     , count(r.*) FILTER (WHERE r.priority_cd = 'Y')  AS PRIORITIES
     , count(e.*) + count(r.*) FILTER (WHERE r.priority_cd = 'Y')   AS TOTAL                  
FROM events e,
requests r
WHERE e.user_id != 1
AND r.id = e.nr_id
AND date(e.event_dt at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '6 months')
AND e.state_cd in ('APPROVED','REJECTED','CONDITIONAL','CANCELLED')
group by e.user_id
order by e.user_id;


In [ ]:
edt = stat_six_month_completed.DataFrame()
edt['examiner'] = edt['examiner'].str.replace('idir/','')

In [ ]:
edt['approved_%'] = ((edt.approved + edt.conditional) / edt.total * 100).round(1)
edt['rejected_%'] = (edt.rejected / edt.total * 100).round(1)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(HTML(edt.to_html()))
#     print('grand total', edt['total'].sum())

Save to CSV

In [ ]:
filename = 'six_month_totals_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'
edt.to_csv(filename, sep=',', encoding='utf-8', index=False)